# Adding Average Route Time (actual_elapsed_time)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import sklearn.preprocessing

In [2]:
my_path = "C:/Users/user/lighthouse/Midterm_Project/Predicting_Flight_Delays/workbench/data/"
df = pd.read_csv(my_path +'flights.csv')
df.head()

,fl_date,mkt_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,carrier,month,day_of_week,haul_length,route,flight_delayed
0,2018-05-04,DL,N754EV,BOS,"Boston, MA",RIC,"Richmond, VA",1455,1453.0,-2.0,...,0.0,0.0,0.0,0.0,Delta Air Lines,5,4,short,BOS-RIC,0
1,2019-12-09,UA,N828UA,MSY,"New Orleans, LA",EWR,"Newark, NJ",1515,1757.0,162.0,...,0.0,178.0,0.0,0.0,United Airlines,12,0,short,MSY-EWR,1
2,2019-12-20,AA,N763US,CLT,"Charlotte, NC",SAT,"San Antonio, TX",1610,1630.0,20.0,...,0.0,0.0,0.0,0.0,American Airlines,12,4,short,CLT-SAT,1
3,2019-07-14,DL,N961DL,ATL,"Atlanta, GA",PWM,"Portland, ME",2114,2114.0,0.0,...,0.0,0.0,0.0,0.0,Delta Air Lines,7,6,short,ATL-PWM,0
4,2019-04-08,AA,N679AW,PHX,"Phoenix, AZ",BOI,"Boise, ID",2021,2015.0,-6.0,...,0.0,0.0,0.0,0.0,American Airlines,4,0,short,PHX-BOI,0


In [3]:
df['dest']=LabelEncoder().fit_transform(df.dest)
df['origin']=LabelEncoder().fit_transform(df.origin)
df['carrier_encoded']=LabelEncoder().fit_transform(df.mkt_unique_carrier)
df['route']=LabelEncoder().fit_transform(df.route)
df['haul_encoded']=LabelEncoder().fit_transform(df.haul_length)
bins = [0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400]
df['binned_dep'] = np.searchsorted(bins, df['crs_dep_time'].values)
df['binned_arr'] = np.searchsorted(bins, df['crs_arr_time'].values)
bins = [0, 1100, 4100, 10000]
df['binned_distance'] = np.searchsorted(bins, df['distance'].values)

#average elapsed time
df['flight_carrier'] = df['mkt_unique_carrier']
route_time_grouped = df.groupby(by='flight_carrier').mean()
route_time_grouped.reset_index(inplace=True)
route_time_grouped['average_route_time'] = route_time_grouped['actual_elapsed_time']
route_time_grouped = route_time_grouped[['flight_carrier','average_route_time']]
df = df.merge(route_time_grouped, on = 'flight_carrier', how = 'outer')
df

,fl_date,mkt_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,...,haul_length,route,flight_delayed,carrier_encoded,haul_encoded,binned_dep,binned_arr,binned_distance,flight_carrier,average_route_time
0,2018-05-04,DL,N754EV,46,"Boston, MA",297,"Richmond, VA",1455,1453.0,-2.0,...,short,672,0,3,2,5,6,1,DL,125.024174
1,2019-07-14,DL,N961DL,21,"Atlanta, GA",290,"Portland, ME",2114,2114.0,0.0,...,short,255,0,3,2,8,8,1,DL,125.024174
2,2018-03-17,DL,N8836A,21,"Atlanta, GA",83,"Columbus, GA",1001,954.0,-7.0,...,short,167,1,3,2,4,4,1,DL,125.024174
3,2018-09-10,DL,N718TW,184,"New York, NY",320,"San Francisco, CA",700,700.0,0.0,...,medium,2941,0,3,1,3,4,2,DL,125.024174
4,2019-08-14,DL,N8943A,86,"Mosinee, WI",240,"Minneapolis, MN",1546,1558.0,12.0,...,short,1249,1,3,2,6,6,1,DL,125.024174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195867,2018-01-22,VX,N642VA,195,"Los Angeles, CA",318,"Seattle, WA",1500,1453.0,-7.0,...,short,3191,0,9,2,5,6,1,VX,208.322115
195868,2018-01-14,VX,N922VA,193,"Las Vegas, NV",320,"San Francisco, CA",1050,1039.0,-11.0,...,short,3086,0,9,2,4,5,1,VX,208.322115
195869,2018-03-27,VX,N521VA,195,"Los Angeles, CA",117,"Newark, NJ",1355,1346.0,-9.0,...,medium,3130,0,9,1,5,8,2,VX,208.322115
195870,2018-03-22,VX,N621VA,126,"Fort Lauderdale, FL",320,"San Francisco, CA",1850,1955.0,65.0,...,long,2205,1,9,0,7,8,2,VX,208.322115


In [4]:
numeric_features = [i for i in df.columns if df[i].dtypes!='O']
print(numeric_features)
num_df = df[numeric_features]
final_num_df = num_df.copy()

['origin', 'dest', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'month', 'day_of_week', 'route', 'flight_delayed', 'carrier_encoded', 'haul_encoded', 'binned_dep', 'binned_arr', 'binned_distance', 'average_route_time']


In [5]:
final_num_df = final_num_df.filter(['origin', 'dest','taxi_out', 'taxi_in','month','day_of_week', 'carrier_encoded', 
                                    'haul_encoded', 'binned_dep', 'binned_arr', 'binned_distance', 'route','average_route_time','arr_delay'])

In [6]:
final_num_df.head()

,origin,dest,taxi_out,taxi_in,month,day_of_week,carrier_encoded,haul_encoded,binned_dep,binned_arr,binned_distance,route,average_route_time,arr_delay
0,46,297,20.0,4.0,5,4,3,2,5,6,1,672,125.024174,-14.0
1,21,290,12.0,2.0,7,6,3,2,8,8,1,255,125.024174,-14.0
2,21,83,34.0,3.0,3,5,3,2,4,4,1,167,125.024174,13.0
3,184,320,22.0,8.0,9,0,3,1,3,4,2,2941,125.024174,-18.0
4,86,240,25.0,2.0,8,2,3,2,6,6,1,1249,125.024174,11.0


In [7]:
final_num_df.to_csv('feature_2.csv')